In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sklearn

## 데이터 로드 및 train test로 분할

In [2]:
df_kookmin = pd.read_csv('/Users/lhs2/Desktop/GitHub/MLT/EDA/non_financial/crawling/news/oasis_kookmin.csv') #.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link'])
df_kyunghyan = pd.read_csv('/Users/lhs2/Desktop/GitHub/MLT/EDA/non_financial/crawling/news/oasis_kyunghyan.csv') #.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link'])



In [3]:
df = pd.concat([df_kookmin, df_kyunghyan]).drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','links'])
# df = df.set_index(['dates'])

In [4]:
df

,titles,dates,contents
0,"[IPO 앞둔 오아시스마켓, 매장 간판서 ‘생협’ 뺀다]",2022-04-27 04:06,[\n\n우리소비자생활협동조합의 위탁판매점이라는 표시를 한 오아시스 매장 모습. 오...
1,"[10년간 이어진 ‘생협’ 명칭 논란…오아시스, 간판에서 ‘생협’ 뗀다]",2022-04-26 17:20,"[\n\n우리생협 오아시스 신촌점(왼쪽)과 오아시스 주엽역점. 우리생협, 오아시스 ..."
2,[1시간 내 장보기 서비스 ‘요마트’ 재가동],2022-05-18 04:06,[\n\nGS리테일과 요기요가 17일 전국 즉시 장보기 서비스 ‘요마트’를 오픈했다...
3,"[요기요+GS리테일, ‘요마트’ 부활… ‘퀵커머스’ 치열한 경쟁]",2022-05-17 17:11,[\n\nGS리테일과 요기요가 17일 전국 즉시 장보기 서비스 ‘요마트’를 오픈했다...
4,[엔데믹 임박… 잘나가던 새벽배송 ‘탈진’],2022-04-18 04:06,[\n\nBGF는 헬로네이처에서 운영하던 새벽배송 서비스를 다음 달 말에 종료한다고...
5,"[“출혈경쟁 못 견뎌” 롯데온, BGF 새벽배송 접는다]",2022-04-17 17:22,[\n\nBGF가 운영하는 새벽배송 업체 '헬로네이처'. BGF 제공롯데온에 이어 ...
6,[2차전지 소재 유망기업 대구 유치…1530억원 투자],2022-03-27 13:18,[\n\n대구국가산단 조감도. 대구시 제공대구시는 대구국가산업단지에 2차전지 소재 ...
7,"[SSG닷컴, “내년 증권시장 상장 목표”…대표 주간사 선정]",2021-10-27 16:23,[\n\nSSG닷컴이 기업공개(IPO)를 위해 미래에셋증권과 씨티그룹글로벌마켓증권을...
8,"[마켓컬리, SSG, 오아시스마켓 물류센터에 산업재해 ‘급증’]",2022.08.15 10:22,[물류창고의 좁은 통로에서는 등 뒤로 지게차가 지나다녔다. 지게차의 지게 발을 피하...
9,[IPO 혹한기 속 누가 웃을까…컬리·11번가·오아시스 상장 시동],2022.08.29 16:26,[주요 e커머스 플랫폼이 기업공개(IPO) 레이스에 시동을 걸었다. 컬리를 시작으로...


In [5]:
for i in range(28):
    df.dates[i] =df.dates[i][:10]

In [6]:
df['dates'] = df['dates'].str.replace('.','-')
df['dates'] = df['dates'].str.replace(' ', '')

/var/folders/62/p7jd9w0n1n3d19j8s3z7k3lw0000gp/T/ipykernel_2382/3561615856.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['dates'] = df['dates'].str.replace('.','-')


In [7]:
df = df.sort_values(by=['dates'], axis=0).reset_index().drop(columns=['index'])#.drop(columns=['level_0'])
df

,titles,dates,contents
0,"[SSG닷컴, “내년 증권시장 상장 목표”…대표 주간사 선정]",2021-10-27,[\n\nSSG닷컴이 기업공개(IPO)를 위해 미래에셋증권과 씨티그룹글로벌마켓증권을...
1,[새벽배송 1호 상장사 어디?…SSG닷컴·컬리·오아시스 IPO 경쟁],2021-10-31,[‘새벽배송’을 대표하는 기업들이 최근 경쟁적으로 상장 절차에 돌입하면서 ‘1호 상...
2,[새벽배송 ‘1호 상장사’ 주인공은 누가 될까?],2021-10-31,[마켓컬리·SSG닷컴·오아시스마켓내년 국내 증시 상장을 목표로대표 주간사 선정 잇따...
3,[내년 IPO 규모 30조원대...역대급을 넘어서는 역대급],2021-12-29,[올해 역대 최대를 기록했던 증권 시장의 기업공개(IPO) 규모가 2022년에는 더...
4,[예상 가치 수조원대 기업 ‘줄줄이’…내년 IPO ‘신기록 행진’ 이어지나],2021-12-29,[1월 상장 추진 LG에너지솔루션시총 목표 70조 ‘최대어’ 1순위현대엔지니어링 최...
5,"[네이버·홈쇼핑 등 새벽배송 진출 러쉬…""온라인 장보기 접수""]",2022-03-08,[온라인 장보기 일상화로 신선식품 새벽배송 시장에 신규 사업자들이 잇따라 진출하고 ...
6,[후발주자 잇단 참전…치열해진 새벽배송],2022-03-08,"[네이버, SSG닷컴과 연계해 진출온라인몰인 G마켓·인터파크에CJ온스타일 등 홈쇼핑..."
7,[2차전지 소재 유망기업 대구 유치…1530억원 투자],2022-03-27,[\n\n대구국가산단 조감도. 대구시 제공대구시는 대구국가산업단지에 2차전지 소재 ...
8,[직원 스마트폰 앱 ‘오아시스 루트’…집화·포장·배송 최적화 ‘흑자 루트’],2022-04-10,"[새벽 식품 배송업계서 유일 이익오아시스마켓 물류센터 가보니, 2018년 진출…작년..."
9,"[“출혈경쟁 못 견뎌” 롯데온, BGF 새벽배송 접는다]",2022-04-17,[\n\nBGF가 운영하는 새벽배송 업체 '헬로네이처'. BGF 제공롯데온에 이어 ...


In [8]:
# index 날짜로 변경

df = df.set_index('dates')
df

,titles,contents
dates,,
2021-10-27,"[SSG닷컴, “내년 증권시장 상장 목표”…대표 주간사 선정]",[\n\nSSG닷컴이 기업공개(IPO)를 위해 미래에셋증권과 씨티그룹글로벌마켓증권을...
2021-10-31,[새벽배송 1호 상장사 어디?…SSG닷컴·컬리·오아시스 IPO 경쟁],[‘새벽배송’을 대표하는 기업들이 최근 경쟁적으로 상장 절차에 돌입하면서 ‘1호 상...
2021-10-31,[새벽배송 ‘1호 상장사’ 주인공은 누가 될까?],[마켓컬리·SSG닷컴·오아시스마켓내년 국내 증시 상장을 목표로대표 주간사 선정 잇따...
2021-12-29,[내년 IPO 규모 30조원대...역대급을 넘어서는 역대급],[올해 역대 최대를 기록했던 증권 시장의 기업공개(IPO) 규모가 2022년에는 더...
2021-12-29,[예상 가치 수조원대 기업 ‘줄줄이’…내년 IPO ‘신기록 행진’ 이어지나],[1월 상장 추진 LG에너지솔루션시총 목표 70조 ‘최대어’ 1순위현대엔지니어링 최...
2022-03-08,"[네이버·홈쇼핑 등 새벽배송 진출 러쉬…""온라인 장보기 접수""]",[온라인 장보기 일상화로 신선식품 새벽배송 시장에 신규 사업자들이 잇따라 진출하고 ...
2022-03-08,[후발주자 잇단 참전…치열해진 새벽배송],"[네이버, SSG닷컴과 연계해 진출온라인몰인 G마켓·인터파크에CJ온스타일 등 홈쇼핑..."
2022-03-27,[2차전지 소재 유망기업 대구 유치…1530억원 투자],[\n\n대구국가산단 조감도. 대구시 제공대구시는 대구국가산업단지에 2차전지 소재 ...
2022-04-10,[직원 스마트폰 앱 ‘오아시스 루트’…집화·포장·배송 최적화 ‘흑자 루트’],"[새벽 식품 배송업계서 유일 이익오아시스마켓 물류센터 가보니, 2018년 진출…작년..."


## 같은 날짜 한줄에 표시

In [9]:
df = df.groupby('dates').sum()
df

,titles,contents
dates,,
2021-10-27,"[SSG닷컴, “내년 증권시장 상장 목표”…대표 주간사 선정]",[\n\nSSG닷컴이 기업공개(IPO)를 위해 미래에셋증권과 씨티그룹글로벌마켓증권을...
2021-10-31,[새벽배송 1호 상장사 어디?…SSG닷컴·컬리·오아시스 IPO 경쟁][새벽배송 ‘1...,[‘새벽배송’을 대표하는 기업들이 최근 경쟁적으로 상장 절차에 돌입하면서 ‘1호 상...
2021-12-29,[내년 IPO 규모 30조원대...역대급을 넘어서는 역대급][예상 가치 수조원대 기...,[올해 역대 최대를 기록했던 증권 시장의 기업공개(IPO) 규모가 2022년에는 더...
2022-03-08,"[네이버·홈쇼핑 등 새벽배송 진출 러쉬…""온라인 장보기 접수""][후발주자 잇단 참전...",[온라인 장보기 일상화로 신선식품 새벽배송 시장에 신규 사업자들이 잇따라 진출하고 ...
2022-03-27,[2차전지 소재 유망기업 대구 유치…1530억원 투자],[\n\n대구국가산단 조감도. 대구시 제공대구시는 대구국가산업단지에 2차전지 소재 ...
2022-04-10,[직원 스마트폰 앱 ‘오아시스 루트’…집화·포장·배송 최적화 ‘흑자 루트’],"[새벽 식품 배송업계서 유일 이익오아시스마켓 물류센터 가보니, 2018년 진출…작년..."
2022-04-17,"[“출혈경쟁 못 견뎌” 롯데온, BGF 새벽배송 접는다]",[\n\nBGF가 운영하는 새벽배송 업체 '헬로네이처'. BGF 제공롯데온에 이어 ...
2022-04-18,[엔데믹 임박… 잘나가던 새벽배송 ‘탈진’],[\n\nBGF는 헬로네이처에서 운영하던 새벽배송 서비스를 다음 달 말에 종료한다고...
2022-04-26,"[10년간 이어진 ‘생협’ 명칭 논란…오아시스, 간판에서 ‘생협’ 뗀다]","[\n\n우리생협 오아시스 신촌점(왼쪽)과 오아시스 주엽역점. 우리생협, 오아시스 ..."


## null 값 확인

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 2021-10-27 to 2022-08-29
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   titles    19 non-null     object
 1   contents  19 non-null     object
dtypes: object(2)
memory usage: 456.0+ bytes


## 특수문자 제거 정규표현식

In [11]:
df['titles'] = df['titles'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
df['contents'] = df['contents'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')

/var/folders/62/p7jd9w0n1n3d19j8s3z7k3lw0000gp/T/ipykernel_2382/1747014847.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['titles'] = df['titles'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
/var/folders/62/p7jd9w0n1n3d19j8s3z7k3lw0000gp/T/ipykernel_2382/1747014847.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['contents'] = df['contents'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')


In [12]:
df

,titles,contents
dates,,
2021-10-27,SSG닷컴 내년 증권시장 상장 목표대표 주간사 선정,\n\nSSG닷컴이 기업공개IPO를 위해 미래에셋증권과 씨티그룹글로벌마켓증권을 대표...
2021-10-31,새벽배송 1호 상장사 어디SSG닷컴컬리오아시스 IPO 경쟁새벽배송 1호 상장사 주인...,새벽배송을 대표하는 기업들이 최근 경쟁적으로 상장 절차에 돌입하면서 1호 상장사 타...
2021-12-29,내년 IPO 규모 30조원대역대급을 넘어서는 역대급예상 가치 수조원대 기업 줄줄이내...,올해 역대 최대를 기록했던 증권 시장의 기업공개IPO 규모가 2022년에는 더욱 커...
2022-03-08,네이버홈쇼핑 등 새벽배송 진출 러쉬온라인 장보기 접수후발주자 잇단 참전치열해진 새벽배송,온라인 장보기 일상화로 신선식품 새벽배송 시장에 신규 사업자들이 잇따라 진출하고 있...
2022-03-27,2차전지 소재 유망기업 대구 유치1530억원 투자,\n\n대구국가산단 조감도 대구시 제공대구시는 대구국가산업단지에 2차전지 소재 유망...
2022-04-10,직원 스마트폰 앱 오아시스 루트집화포장배송 최적화 흑자 루트,새벽 식품 배송업계서 유일 이익오아시스마켓 물류센터 가보니 2018년 진출작년 영업...
2022-04-17,출혈경쟁 못 견뎌 롯데온 BGF 새벽배송 접는다,\n\nBGF가 운영하는 새벽배송 업체 헬로네이처 BGF 제공롯데온에 이어 BGF까...
2022-04-18,엔데믹 임박 잘나가던 새벽배송 탈진,\n\nBGF는 헬로네이처에서 운영하던 새벽배송 서비스를 다음 달 말에 종료한다고 ...
2022-04-26,10년간 이어진 생협 명칭 논란오아시스 간판에서 생협 뗀다,\n\n우리생협 오아시스 신촌점왼쪽과 오아시스 주엽역점 우리생협 오아시스 제공새벽배...


## 토큰화

In [13]:
okt = Okt()


In [14]:
# 불용어 처리 

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','\n','\n ', '제목']
stopwords

['의',
 '가',
 '이',
 '은',
 '들',
 '는',
 '좀',
 '잘',
 '걍',
 '과',
 '도',
 '를',
 '으로',
 '자',
 '에',
 '와',
 '한',
 '하다',
 '\n',
 '\n ',
 '제목']

In [15]:
token = []
for sentence in tqdm(df['titles']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    token.append(stopwords_removed_sentence)
    

100%|███████████████████████████████████████████| 19/19 [00:01<00:00, 11.30it/s]


In [16]:
len(token)

19

In [17]:
from collections import Counter

In [19]:
# 한 날짜? 한 행당 횟수 추출

for i in range(19):
    vocab = Counter(token[i])
    print(vocab)

Counter({'SSG': 1, '닷컴': 1, '내년': 1, '증권': 1, '시장': 1, '상장': 1, '목표': 1, '대표': 1, '주': 1, '간사': 1, '선정': 1})
Counter({'새벽': 2, '배송': 2, '1': 2, '호': 2, '상': 2, '장사': 2, '어디': 1, 'SSG': 1, '닷': 1, '컴': 1, '컬리': 1, '오아시스': 1, 'IPO': 1, '경쟁': 1, '주인공': 1, '누가': 1, '되다': 1})
Counter({'내년': 2, 'IPO': 2, '규모': 1, '30조원': 1, '대역대급을': 1, '넘어서다': 1, '역대': 1, '급': 1, '예상': 1, '가치': 1, '수조': 1, '원대': 1, '기업': 1, '줄줄이': 1, '신기록': 1, '행진': 1, '이어지다': 1})
Counter({'새벽': 2, '배송': 2, '네이버': 1, '홈쇼핑': 1, '등': 1, '진출': 1, '러쉬': 1, '온라인': 1, '장보기': 1, '접수': 1, '후': 1, '발주': 1, '잇다': 1, '참전': 1, '치열하다': 1})
Counter({'2': 1, '차다': 1, '소재': 1, '유망': 1, '기업': 1, '대구': 1, '유치': 1, '1530억원': 1, '투자': 1})
Counter({'루트': 2, '직원': 1, '스마트폰': 1, '앱': 1, '오아시스': 1, '집': 1, '화': 1, '포장': 1, '배송': 1, '최적화': 1, '흑자': 1})
Counter({'출혈': 1, '경쟁': 1, '못': 1, '견디다': 1, '롯데': 1, '온': 1, 'BGF': 1, '새벽': 1, '배송': 1, '접다': 1})
Counter({'엔데믹': 1, '임박': 1, '잘나가다': 1, '새벽': 1, '배송': 1, '탈진': 1})
Counter({'생협': 2, '10년': 1, '간': 

In [20]:
# 전체 중 빈도수 출력

all_word_list = sum(token, [])
vocab = Counter(all_word_list)
print(vocab)

Counter({'배송': 11, '새벽': 10, '오아시스': 8, '시장': 5, 'IPO': 5, '경쟁': 4, '마켓': 4, '물류': 4, '센터': 4, 'SSG': 3, '내년': 3, '대표': 3, '컬리': 3, '네이버': 3, '생협': 3, '요': 3, '재': 3, '상장': 2, '1': 2, '호': 2, '상': 2, '장사': 2, '누가': 2, '기업': 2, '등': 2, '장보기': 2, '치열하다': 2, '루트': 2, '출혈': 2, '간판': 2, '위상': 2, '플랫폼': 2, '재벌': 2, '총수': 2, '마트': 2, '펴다': 2, '나': 2, '유통': 2, '강': 2, '백': 2, '기': 2, '확대': 2, '속': 2, '노동자': 2, '짜내다': 2, '21': 2, '세기': 2, '평화시장': 2, '불': 2, '안정': 2, '노동': 2, '핵심': 2, '닷컴': 1, '증권': 1, '목표': 1, '주': 1, '간사': 1, '선정': 1, '어디': 1, '닷': 1, '컴': 1, '주인공': 1, '되다': 1, '규모': 1, '30조원': 1, '대역대급을': 1, '넘어서다': 1, '역대': 1, '급': 1, '예상': 1, '가치': 1, '수조': 1, '원대': 1, '줄줄이': 1, '신기록': 1, '행진': 1, '이어지다': 1, '홈쇼핑': 1, '진출': 1, '러쉬': 1, '온라인': 1, '접수': 1, '후': 1, '발주': 1, '잇다': 1, '참전': 1, '2': 1, '차다': 1, '소재': 1, '유망': 1, '대구': 1, '유치': 1, '1530억원': 1, '투자': 1, '직원': 1, '스마트폰': 1, '앱': 1, '집': 1, '화': 1, '포장': 1, '최적화': 1, '흑자': 1, '못': 1, '견디다': 1, '롯데': 1, '온': 1, 'BGF': 1, '접다': 1, '엔데믹

In [21]:
word_list = list(vocab.keys())
word_list

['SSG',
 '닷컴',
 '내년',
 '증권',
 '시장',
 '상장',
 '목표',
 '대표',
 '주',
 '간사',
 '선정',
 '새벽',
 '배송',
 '1',
 '호',
 '상',
 '장사',
 '어디',
 '닷',
 '컴',
 '컬리',
 '오아시스',
 'IPO',
 '경쟁',
 '주인공',
 '누가',
 '되다',
 '규모',
 '30조원',
 '대역대급을',
 '넘어서다',
 '역대',
 '급',
 '예상',
 '가치',
 '수조',
 '원대',
 '기업',
 '줄줄이',
 '신기록',
 '행진',
 '이어지다',
 '네이버',
 '홈쇼핑',
 '등',
 '진출',
 '러쉬',
 '온라인',
 '장보기',
 '접수',
 '후',
 '발주',
 '잇다',
 '참전',
 '치열하다',
 '2',
 '차다',
 '소재',
 '유망',
 '대구',
 '유치',
 '1530억원',
 '투자',
 '직원',
 '스마트폰',
 '앱',
 '루트',
 '집',
 '화',
 '포장',
 '최적화',
 '흑자',
 '출혈',
 '못',
 '견디다',
 '롯데',
 '온',
 'BGF',
 '접다',
 '엔데믹',
 '임박',
 '잘나가다',
 '탈진',
 '10년',
 '간',
 '이어진',
 '생협',
 '명칭',
 '논란',
 '간판',
 '에서',
 '떼다',
 '앞두다',
 '마켓',
 '매장',
 '서',
 '빼다',
 '위상',
 '높아지다',
 '플랫폼',
 '재벌',
 '총수',
 '한자리',
 '달라지다',
 '실감',
 '어깨',
 '나란하다',
 '요기',
 '요',
 'GS',
 '리테일',
 '마트',
 '부활',
 '퀵',
 '커머스',
 '1시간',
 '내',
 '서비스',
 '재',
 '가동',
 '이랜드',
 '맞다',
 '손',
 '신선',
 '판도',
 '바꾸다',
 '것',
 '여기다',
 '때',
 '몸값',
 '1조',
 '유니콘',
 '5',
 '개',
 '늘다',
 '펴다',
 '나',
 '유통',
 '강',
 '

In [22]:
word_rank = list(vocab.values())
word_rank

[3,
 1,
 3,
 1,
 5,
 2,
 1,
 3,
 1,
 1,
 1,
 10,
 11,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 3,
 8,
 5,
 4,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 3,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1]

In [23]:
data = {'word': word_list, 'count' : word_rank}
word_data = pd.DataFrame.from_dict(data)
word_data

,word,count
0,SSG,3
1,닷컴,1
2,내년,3
3,증권,1
4,시장,5
...,...,...
184,한기,1
185,웃다,1
186,11,1
187,번가,1


In [24]:
word_data = word_data.sort_values(by=['count'], ascending=False).reset_index().drop(columns=['index'])

In [25]:
word_data

,word,count
0,배송,11
1,새벽,10
2,오아시스,8
3,시장,5
4,IPO,5
...,...,...
184,온,1
185,BGF,1
186,접다,1
187,엔데믹,1


In [26]:
for i in range(19):
    print(word_data[ i*10 : (i+1)*10 ])

   word  count
0    배송     11
1    새벽     10
2  오아시스      8
3    시장      5
4   IPO      5
5    마켓      4
6    물류      4
7    센터      4
8    경쟁      4
9   SSG      3
   word  count
10   컬리      3
11    재      3
12   생협      3
13  네이버      3
14    요      3
15   대표      3
16   내년      3
17    상      2
18   유통      2
19   장사      2
   word  count
20   루트      2
21    속      2
22   확대      2
23    기      2
24    백      2
25   출혈      2
26    강      2
27    나      2
28    등      2
29   펴다      2
    word  count
30    상장      2
31    간판      2
32    위상      2
33   플랫폼      2
34    재벌      2
35    총수      2
36    마트      2
37   장보기      2
38  치열하다      2
39    세기      2
    word  count
40    안정      2
41  평화시장      2
42    21      2
43   짜내다      2
44   노동자      2
45     불      2
46     1      2
47     호      2
48    누가      2
49    기업      2
   word  count
50   노동      2
51   핵심      2
52    개      1
53   부활      1
54    퀵      1
55  커머스      1
56    5      1
57  유니콘      1
58   1조      1
59 

In [27]:
# word_data.to_csv('sentimental(news)_.csv')

In [29]:
sentimental = pd.read_csv('sentimental(news).csv').drop(columns=['Unnamed: 0'])
sentimental = sentimental.fillna('-')

In [31]:
sentimental.to_csv('sentimental(news).csv')

### 추출했던 word_list에서 불용어 추가

In [ ]:
import csv

with open("stopword.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(word_list)

In [ ]:
with open('stopword.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

In [ ]:
stopword_add = sum(data, [])
stopword_add